<a href="https://colab.research.google.com/github/S-Kitaev/TimeSeriesPredictions/blob/main/exp1_statistics_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>